## Building flask api

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.chdir('../../../')

In [3]:
import numpy as np
from pathlib import Path
import pandas as pd
import hashlib
import shutil
import json

In [4]:
np.set_printoptions(edgeitems=10, threshold=40, linewidth=200)

In [5]:
data_dir = Path('data/midi/v20/')
path = data_dir/'piano_duet'

In [6]:
encoded_path = Path('data/midi/examples')

## Creating Subset

In [7]:
song_ids = [
    '99c5db9ce936b7a1c5faab57cc3f160c',
    'aeee134e4034e5f98bb630c56d2f7f8c',
    'b4c62f3d11f448db69c4fa15e27e9e8e',
    '8f10995e036c5f590ca616dee0c9c7d7',
    '58890c433f173ba77220274017b8af92',
    '19762ebd7059dd5f8809f98b2575d8c1',
    'c4d9e72a92f931b2c72e84b6efee4b51',
    'de1fa62a9c209491d4fd81f1d1ceb55e',
    'f8b65d3624d9a68e417792ec27de9296',
    '2ad86f3ecd71b058eac9cc5ceba11db8',
    '999e8169b01f59fa28e63b142f35feda',
    '7f0f162d111f880cf77ff9bed15acd5d',
    '077b40997776b84366ee18056948ae38',
    '065b1c94fe8545bbe8eeec2f1bf4efa2',
    'f2062929514d3084f73a1ad81bbbe01f',
    '68ce7fc929f395291507cc47510cd82c',
    '6ae7e284e78824ee4c17dc6e6bda361c',
    '73b2b8e7db746b7e4b6fc3fa4658671b',
    '3cc8e692016fb1d8c8f097e259ea1ea5',
    
    'b51d911631b67babe1612dae455600e3',
    '81efc618ff11aa17d4e96219380c115e',
    '583aee3ac9d91125102096d28926702b',
    'e250d1faf9b3265f23c049d113f8a773',
    'e8820e3b81d2429d684c986ce2f57071',
    '83ee71fe7fef0bd1fd15a49740cfc967',
    '1b341ae44d35a6c8d21f6ba3af65dd97',
    '4296f489066d02db3491bcde78667746',
    '8c0b6ff7e893dca13edd90a4d9ebb892',
    'c8bcf08c8d31c98c61a91b2d90a9cdde',
    '50b4cbff1b6c2886c31332e5fb1a5f56',
    'bb3324a9068db739eff1b2fc1cc0670b',
    
]

# song_ids = [sid[::-1] for sid in song_ids]

In [12]:
df = pd.read_csv(path/'piano_duet.csv')
df = df.loc[df['numpy'].notna()]
df = df.loc[df.source == 'hooktheory_c']
df = df.where((pd.notnull(df)), None) # nan values break json
df.shape

/home/ubuntu/anaconda3/envs/musicautobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,3,4,7,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(20028, 16)

In [13]:
subset = df.loc[df.md5.isin(song_ids)]; subset

,parts,ht_time_signature,midi,section,midi_title,title,numpy,mxl,ht_bpm,md5,source,artist,ht_key,ht_mode,genres,song_url
177357,verse,4.0,midi_sources/hooktheory/pianoroll/w/wolfgang-a...,verse,Symphony no 40 in G minor - I,symphony-no-40-in-g-minor---i,piano_duet/hooktheory/pianoroll/w/wolfgang-ama...,None,115.0,4296f489066d02db3491bcde78667746,hooktheory_c,wolfgang-amadeus-mozart,C,1.0,Classical,https://www.hooktheory.com/theorytab/view/wolf...
178513,"intro,verse,chorus",4.0,midi_sources/hooktheory/pianoroll/j/jackson-5/...,intro,I Want You Back,i-want-you-back,piano_duet/hooktheory/pianoroll/j/jackson-5/i-...,None,98.0,2ad86f3ecd71b058eac9cc5ceba11db8,hooktheory_c,jackson-5,C,1.0,None,https://www.hooktheory.com/theorytab/view/jack...
178716,"intro,pre-chorus,chorus",4.0,midi_sources/hooktheory/pianoroll/j/johann-pac...,chorus,Pachelbel Canon Chorus,canon-in-d-major,piano_duet/hooktheory/pianoroll/j/johann-pache...,None,138.0,c4d9e72a92f931b2c72e84b6efee4b51,hooktheory_c,johann-pachelbel,C,1.0,Classical,https://www.hooktheory.com/theorytab/view/joha...
179774,"intro,solo",4.0,midi_sources/hooktheory/pianoroll/l/led-zeppel...,intro,Stairway to Heaven,stairway-to-heaven,piano_duet/hooktheory/pianoroll/l/led-zeppelin...,None,74.0,8c0b6ff7e893dca13edd90a4d9ebb892,hooktheory_c,led-zeppelin,C,1.0,Blues,https://www.hooktheory.com/theorytab/view/led-...
179918,"verse,chorus,bridge,solo",3.0,midi_sources/hooktheory/pianoroll/l/ludwig-van...,verse,Fuer Elise,fuer-elise,piano_duet/hooktheory/pianoroll/l/ludwig-van-b...,None,139.0,6ae7e284e78824ee4c17dc6e6bda361c,hooktheory_c,ludwig-van-beethoven,C,1.0,Classical,https://www.hooktheory.com/theorytab/view/ludw...
180059,"verse,chorus,chorus-lead-out,bridge",4.0,midi_sources/hooktheory/pianoroll/l/lady-gaga/...,chorus,Bad Romance,bad-romance,piano_duet/hooktheory/pianoroll/l/lady-gaga/ba...,None,119.0,bb3324a9068db739eff1b2fc1cc0670b,hooktheory_c,lady-gaga,C,1.0,Pop,https://www.hooktheory.com/theorytab/view/lady...
180184,"pre-chorus-and-chorus,chorus",4.0,midi_sources/hooktheory/pianoroll/l/linkin-par...,chorus,In The End,in-the-end,piano_duet/hooktheory/pianoroll/l/linkin-park/...,None,107.0,b51d911631b67babe1612dae455600e3,hooktheory_c,linkin-park,C,1.0,Blues,https://www.hooktheory.com/theorytab/view/link...
180907,"intro,verse-and-pre-chorus,chorus",4.0,midi_sources/hooktheory/pianoroll/v/vanessa-ca...,verse-and-pre-chorus,A thousand miles verse and prechorus,a-thousand-miles,piano_duet/hooktheory/pianoroll/v/vanessa-carl...,None,96.0,58890c433f173ba77220274017b8af92,hooktheory_c,vanessa-carlton,C,1.0,"Pop,Singer-Songwriter",https://www.hooktheory.com/theorytab/view/vane...
182159,chorus,4.0,midi_sources/hooktheory/pianoroll/b/black-eyed...,chorus,Where is the Love Chorus,where-is-the-love,piano_duet/hooktheory/pianoroll/b/black-eyed-p...,None,93.0,68ce7fc929f395291507cc47510cd82c,hooktheory_c,black-eyed-peas,C,1.0,None,https://www.hooktheory.com/theorytab/view/blac...
182319,"intro-and-verse,pre-chorus,chorus",4.0,midi_sources/hooktheory/pianoroll/b/bruno-mars...,chorus,Locked C,locked-out-of-heaven,piano_duet/hooktheory/pianoroll/b/bruno-mars/l...,None,147.0,077b40997776b84366ee18056948ae38,hooktheory_c,bruno-mars,C,1.0,Pop,https://www.hooktheory.com/theorytab/view/brun...


In [14]:
recordlist = subset.to_dict('records')

In [15]:
recordlist

[{'parts': 'verse',
  'ht_time_signature': 4.0,
  'midi': 'midi_sources/hooktheory/pianoroll/w/wolfgang-amadeus-mozart/symphony-no-40-in-g-minor---i/verse_key_cmajor.mid',
  'section': 'verse',
  'midi_title': 'Symphony no 40 in G minor - I',
  'title': 'symphony-no-40-in-g-minor---i',
  'numpy': 'piano_duet/hooktheory/pianoroll/w/wolfgang-amadeus-mozart/symphony-no-40-in-g-minor---i/verse_key_cmajor.npy',
  'mxl': None,
  'ht_bpm': 115.0,
  'md5': '4296f489066d02db3491bcde78667746',
  'source': 'hooktheory_c',
  'artist': 'wolfgang-amadeus-mozart',
  'ht_key': 'C',
  'ht_mode': 1.0,
  'genres': 'Classical',
  'song_url': 'https://www.hooktheory.com/theorytab/view/wolfgang-amadeus-mozart/symphony-no-40-in-g-minor---i'},
 {'parts': 'intro,verse,chorus',
  'ht_time_signature': 4.0,
  'midi': 'midi_sources/hooktheory/pianoroll/j/jackson-5/i-want-you-back/intro_key_cmajor.mid',
  'section': 'intro',
  'midi_title': 'I Want You Back',
  'title': 'i-want-you-back',
  'numpy': 'piano_duet/hoo

## Saving to example Folder

In [16]:
def format_meta(s):
    title = s['title'].title().replace('-', ' ')
    artist = s['artist'].title().replace('-', ' ')
    display = ' - '.join([title, artist])
    if s.get('section'): display += ' - ' + s['section'].title()
        
    key_file = data_dir/s['midi']
    to_file = encoded_path/f'{display}.mid'
    if not to_file.exists():
        to_file.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(str(key_file), str(to_file))
    return to_file

In [17]:
[format_meta(s) for s in recordlist]

[PosixPath('data/midi/examples/Symphony No 40 In G Minor   I - Wolfgang Amadeus Mozart - Verse.mid'),
 PosixPath('data/midi/examples/I Want You Back - Jackson 5 - Intro.mid'),
 PosixPath('data/midi/examples/Canon In D Major - Johann Pachelbel - Chorus.mid'),
 PosixPath('data/midi/examples/Stairway To Heaven - Led Zeppelin - Intro.mid'),
 PosixPath('data/midi/examples/Fuer Elise - Ludwig Van Beethoven - Verse.mid'),
 PosixPath('data/midi/examples/Bad Romance - Lady Gaga - Chorus.mid'),
 PosixPath('data/midi/examples/In The End - Linkin Park - Chorus.mid'),
 PosixPath('data/midi/examples/A Thousand Miles - Vanessa Carlton - Verse-And-Pre-Chorus.mid'),
 PosixPath('data/midi/examples/Where Is The Love - Black Eyed Peas - Chorus.mid'),
 PosixPath('data/midi/examples/Locked Out Of Heaven - Bruno Mars - Chorus.mid'),
 PosixPath('data/midi/examples/Oops I Did It Again - Britney Spears - Chorus.mid'),
 PosixPath('data/midi/examples/Middle - Zedd - Pre-Chorus.mid'),
 PosixPath('data/midi/example

In [18]:
p = Path('../../data/midi/examples/Levels - Avicii - Verse.mid').absolute()
p, p.exists()

(PosixPath('/home/ubuntu/musicautobot/../../data/midi/examples/Levels - Avicii - Verse.mid'),
 False)

## Creating Asset Folder

In [19]:
asset_path = Path('data/assets/examples/seed')

In [20]:
def save_to_path(s):
    title = s['title'].title().replace('-', ' ')
    artist = s['artist'].title().replace('-', ' ')
    display = ' - '.join([title, artist])
    if s.get('section'): display += ' - ' + s['section'].title()
    sid = s['md5']
    
    json_data = {
        'title': title,
        'artist': artist,
        'bpm': s['ht_bpm'],
        'display': display,
        'section': s['section'],
        'genres': s['genres'],
        'sid': sid
    }
    source_file = data_dir/s['midi']
#     key_file = str(source_file).replace('cmajor', 'original')
    key_file = source_file
    to_file = asset_path/f'{sid[::-1]}.mid'
    to_json = asset_path/f'{sid[::-1]}.json'
    if not to_file.exists():
        to_file.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy(str(key_file), str(to_file))
        with open(to_json, 'w') as jp:
            json.dump(json_data, jp)
    
    return json_data

In [23]:
htlist = [save_to_path(s) for s in recordlist]
json_path = asset_path.parent/'json/htlist.json'
json_path.parent.mkdir(parents=True, exist_ok=True)
with open(json_path, 'w') as fp:
    json.dump(htlist, fp, separators=(',', ':'))

In [24]:
htlist

[{'title': 'Symphony No 40 In G Minor   I',
  'artist': 'Wolfgang Amadeus Mozart',
  'bpm': 115.0,
  'display': 'Symphony No 40 In G Minor   I - Wolfgang Amadeus Mozart - Verse',
  'section': 'verse',
  'genres': 'Classical',
  'sid': '4296f489066d02db3491bcde78667746'},
 {'title': 'I Want You Back',
  'artist': 'Jackson 5',
  'bpm': 98.0,
  'display': 'I Want You Back - Jackson 5 - Intro',
  'section': 'intro',
  'genres': None,
  'sid': '2ad86f3ecd71b058eac9cc5ceba11db8'},
 {'title': 'Canon In D Major',
  'artist': 'Johann Pachelbel',
  'bpm': 138.0,
  'display': 'Canon In D Major - Johann Pachelbel - Chorus',
  'section': 'chorus',
  'genres': 'Classical',
  'sid': 'c4d9e72a92f931b2c72e84b6efee4b51'},
 {'title': 'Stairway To Heaven',
  'artist': 'Led Zeppelin',
  'bpm': 74.0,
  'display': 'Stairway To Heaven - Led Zeppelin - Intro',
  'section': 'intro',
  'genres': 'Blues',
  'sid': '8c0b6ff7e893dca13edd90a4d9ebb892'},
 {'title': 'Fuer Elise',
  'artist': 'Ludwig Van Beethoven',
  